# Dev contrastive knowledge assesment notebook for LLaMa

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/CKA_dev_LLaMa.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Dependencies

In [1]:
!git clone https://github.com/daniel-furman/Capstone.git

Cloning into 'Capstone'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (331/331), done.
remote: Compressing objects: 100% (206/206), done.
remote: Total 331 (delta 158), reused 257 (delta 85), pack-reused 0
Receiving objects: 100% (331/331), 21.16 MiB | 31.63 MiB/s, done.
Resolving deltas: 100% (158/158), done.


In [2]:
!pip install -r /content/Capstone/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.3/76.3 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 91.7 MB/s eta 0:00:00


## Imports

In [3]:
import os
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## LLaMa specific setup

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!git clone https://github.com/daniel-furman/transformers_llama.git

Cloning into 'transformers_llama'...
remote: Enumerating objects: 124157, done.
remote: Total 124157 (delta 0), reused 0 (delta 0), pack-reused 124157
Receiving objects: 100% (124157/124157), 123.04 MiB | 36.33 MiB/s, done.
Resolving deltas: 100% (93894/93894), done.


In [7]:
# only need to run when using a model for first time

!python /content/transformers_llama/src/transformers/models/llama/convert_llama_weights_to_hf.py \
    --input_dir '/content/drive/MyDrive/Colab Files/llama/LLaMA/' \
    --model_size 13B \
    --output_dir '/content/drive/MyDrive/Colab Files/llama/LLaMA/int8'

## Run the LLaMas 🦙🦙

In [8]:
os.chdir('/content/transformers_llama')
!pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/transformers_llama
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.27.0.dev0-py3-none-any.whl size=6688316 sha256=ba74fef3c191690073f9be079784d249387a163e0a0c71d8962de877ca442585
  Stored in directory: /tmp/pip-ephem-wheel-cache-2bobo4a7/wheels/86/c1/27/9d840f9c03869c3b830926e03c647575ae0f506156db708a64
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.1
    Uninstalling transformers-4.26.1:
      Successfully uninstalled transformers-4.26.1


In [9]:
import transformers as transformers_LLaMa

In [10]:
tokenizer = transformers_LLaMa.LLaMATokenizer.from_pretrained(
    "/content/drive/MyDrive/Colab Files/llama/LLaMA/int8/tokenizer/"
)

model = transformers_LLaMa.LLaMAForCausalLM.from_pretrained(
    "/content/drive/MyDrive/Colab Files/llama/LLaMA/int8/llama-13b/",
    load_in_8bit=True,
    device_map="auto"
)


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


Loading checkpoint shards:   0%|          | 0/41 [00:00<?, ?it/s]

In [11]:
batch = tokenizer(
    "Steve Jobs is the founder of",
    return_tensors="pt",
    add_special_tokens=False
)

batch = {k: v.cuda() for k, v in batch.items()}
generated = model.generate(batch["input_ids"], max_length=10)
print(tokenizer.decode(generated[0]))

/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


Steve Jobs is the founder of Apple Inc.
